# Quarterly Sales Tax Revenue Analysis

### [Data Source](https://tax.utah.gov/econstats/sales)

### Q1: January, February, March

In [1]:
import pandas as pd
import os
import numpy as np
import arcpy
from arcgis.features import GeoAccessor, GeoSeriesAccessor
arcpy.env.overwriteOutput = True

In [2]:
# show all columns
pd.options.display.max_columns = None

In [3]:
# Removes misc characters from string
def replace(string):
    string = string.replace("&", "")
    string = string.replace("-", " ")
    string = string.replace("(", "")
    string = string.replace(")", "")
    string = string.replace("/", " ")
    string = string.replace(",", "")
    string = string.replace(".", "")
    string = string.replace(" ", "_")
    return string


### Load Q2 2020 sales tax revenue data

In [4]:
# load excel sheet into a dataframe
xlsx = '.\\Inputs\\2020-01-quarterly-sales.xlsx'
q1_2020 = pd.read_excel(xlsx, sheet_name='Table 9', header=5)

# Drop last empty two rows
q1_2020.drop(q1_2020.tail(2).index,inplace=True)

# forward fill values from merged cells
q1_2020['County'] = q1_2020['County'].fillna(method='ffill')
q1_2020['Location Code'] = q1_2020['Location Code'].fillna(method='ffill')
q1_2020['City'] = q1_2020['City'].fillna(method='ffill')

# Figures with less than 10 taxpayers have been rounded up per Tax Commission disclosure rules.
# Rename column and convert values to boolean
q1_2020.rename(columns={'Unnamed: 5':'Rounded Up'}, inplace=True)
crosswalk = {'*':True, np.nan:False} # County_ID
q1_2020['Rounded Up'] = q1_2020['Rounded Up'].map(crosswalk)

q1_2020.head()


,County,Location Code,City,Economic Sector (NAICS Code),2020Q1,Rounded Up
0,Beaver County,1002.0,Beaver City,"AGRICULTURE, FORESTRY, FISHING & HUNTING (11)",30000.0,True
1,Beaver County,1002.0,Beaver City,"MINING, QUARRYING, & OIL & GAS EXTRACTION (21)",3000.0,True
2,Beaver County,1002.0,Beaver City,UTILITIES (22),900000.0,True
3,Beaver County,1002.0,Beaver City,CONSTRUCTION (23),20000.0,True
4,Beaver County,1002.0,Beaver City,MANUFACTURING (31-33),1158075.0,False


In [5]:
# pivot the table to get sectors as columns with tax values by city
sectors_q1_2020 = pd.pivot_table(q1_2020,values = '2020Q1',index ='City', columns = 'Economic Sector (NAICS Code)', aggfunc='first')

# fill NAs with 0
sectors_q1_2020 = sectors_q1_2020.fillna(0)


# fix column names
#new_names = [replace(item) for item in list(sectors_q1_2020.columns)]
new_names = ['L_ACCOMMODATION_20', 'S_ADMIN_SUPPORT_20','A_AG_WILDLIFE_20','L_CULTURAL_REC_20','C_CONSTRUCTION_20',
'E_EDUCATION_20','S_FINANCIAL_20','L_RSTRNT_BAR_20','H_HEALTH_CARE_20','S_IT_DATA_20','S_CORPORATE_MGMT_20',
'M_MANUFACTURING_20','E_EXTRACTION_20','O_OTHER_20','X_ADJUSTMENTS_20','R_AUTO_PRIVATE_20','S_PROF_TECH_SRV_20',
'G_GOVERNMENT_20','S_RENTAL_LEASING_20','R_BUILDING_SUPPLY_20','R_CLOTHING_20','R_ELECTRONICS_20','R_GROCERY_BEV_20',
'R_FURNITURE_20','R_GAS_STATIONS_20','R_GENERAL_RETAIL_20','R_HEALTH_RETAIL_20','R_OTHER_RETAIL_20','R_AUTO_RETAIL_20',
'R_NONSTORE_RETAIL_20','R_SPORT_HOBBY_20','L_SPECIAL_EVENT_20','W_DISTRIBUTION_20','X_UNKNOWN_20','U_UTILITIES_20',
'W_WHLSALE_DURABLE_20','S_WHLSLE_ETRADE_20','W_WHLSLE_NDURABLE_20']
sectors_q1_2020.columns = new_names

# get the total sales tax revenue
sectors_q1_2020['TOTAL_20'] = sectors_q1_2020[new_names].sum(axis=1)

sectors_q1_2020.head()

,L_ACCOMMODATION_20,S_ADMIN_SUPPORT_20,A_AG_WILDLIFE_20,L_CULTURAL_REC_20,C_CONSTRUCTION_20,E_EDUCATION_20,S_FINANCIAL_20,L_RSTRNT_BAR_20,H_HEALTH_CARE_20,S_IT_DATA_20,S_CORPORATE_MGMT_20,M_MANUFACTURING_20,E_EXTRACTION_20,O_OTHER_20,X_ADJUSTMENTS_20,R_AUTO_PRIVATE_20,S_PROF_TECH_SRV_20,G_GOVERNMENT_20,S_RENTAL_LEASING_20,R_BUILDING_SUPPLY_20,R_CLOTHING_20,R_ELECTRONICS_20,R_GROCERY_BEV_20,R_FURNITURE_20,R_GAS_STATIONS_20,R_GENERAL_RETAIL_20,R_HEALTH_RETAIL_20,R_OTHER_RETAIL_20,R_AUTO_RETAIL_20,R_NONSTORE_RETAIL_20,R_SPORT_HOBBY_20,L_SPECIAL_EVENT_20,W_DISTRIBUTION_20,X_UNKNOWN_20,U_UTILITIES_20,W_WHLSALE_DURABLE_20,S_WHLSLE_ETRADE_20,W_WHLSLE_NDURABLE_20,TOTAL_20
City,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
American Fork,461346.0,583528.0,200000.0,1672492.0,8549541.0,77529.0,3192392.0,23701728.0,470936.0,8512406.0,9000.0,3775746.0,80000.0,9961976.0,1819138.0,2000000.0,4266418.0,15000.0,3427382.0,17263059.0,5007232.0,6482038.0,4627911.0,3282013.0,2500000.0,38344973.0,6141802.0,6580936.0,75968679.0,14060388.0,6470057.0,0.0,559259.0,0.0,8250000.0,14284243.0,305397.0,3960874.0,286865419.0
Beaver City,864111.0,22157.0,30000.0,8000.0,20000.0,20000.0,254817.0,2013748.0,60000.0,435101.0,6000.0,1158075.0,3000.0,788673.0,264908.0,350000.0,116728.0,0.0,80776.0,497496.0,45083.0,29368.0,1750000.0,4000.0,3000000.0,476418.0,91441.0,221817.0,658691.0,1275690.0,20572.0,0.0,2000.0,0.0,900000.0,405586.0,19793.0,128131.0,16022180.0
Blanding,350343.0,34733.0,1000.0,25000.0,250000.0,150000.0,22597.0,400000.0,60000.0,441230.0,1000.0,240368.0,0.0,269921.0,-625850.0,600000.0,103492.0,700000.0,60548.0,1307977.0,51653.0,115116.0,2000000.0,3000.0,1250000.0,645883.0,74551.0,107323.0,264369.0,1211254.0,41821.0,0.0,30000.0,0.0,150000.0,303342.0,8000.0,288716.0,10937387.0
Bountiful,200000.0,2097686.0,15000.0,1002730.0,1864898.0,139922.0,1191614.0,12291550.0,1168394.0,6436583.0,30000.0,2642459.0,15000.0,6455142.0,1649557.0,3000000.0,3263809.0,70000.0,1900181.0,1111674.0,2381625.0,3224014.0,22666281.0,1464503.0,2250000.0,1807171.0,2176782.0,2925989.0,42911494.0,14208053.0,1755428.0,0.0,50000.0,1000.0,11250000.0,5343729.0,100630.0,723433.0,161786331.0
Brigham,858416.0,184456.0,3000.0,350000.0,726011.0,52422.0,404093.0,5940942.0,346590.0,2755109.0,8000.0,5284338.0,1000.0,3297982.0,585715.0,2000000.0,694141.0,1000.0,1562099.0,1522381.0,368586.0,1316243.0,11284680.0,531658.0,2882324.0,529048.0,370879.0,1126037.0,12300362.0,5081783.0,365033.0,1000.0,80000.0,0.0,4250000.0,8823583.0,126696.0,283116.0,76298723.0


### Load Q2 2019 sales tax revenue data

In [6]:
# load excel sheet into a dataframe
xlsx = '.\\Inputs\\2019-01-quarterly-sales.xlsx'
q1_2019 = pd.read_excel(xlsx, sheet_name='Table 9', header=5)

# Drop last empty two rows
q1_2019.drop(q1_2019.tail(2).index,inplace=True)

# forward fill values from merged cells
q1_2019['County'] = q1_2019['County'].fillna(method='ffill')
q1_2019['Location Code'] = q1_2019['Location Code'].fillna(method='ffill')
q1_2019['City'] = q1_2019['City'].fillna(method='ffill')

# Figures with less than 10 taxpayers have been rounded up per Tax Commission disclosure rules.
# Rename column and convert values to boolean
q1_2019.rename(columns={'Unnamed: 5':'Rounded Up'}, inplace=True)
crosswalk = {'*':True, np.nan:False} # County_ID
q1_2019['Rounded Up'] = q1_2019['Rounded Up'].map(crosswalk)

q1_2019.tail()

,County,Location Code,City,Economic Sector (NAICS Code),2019Q1,Rounded Up
2393,Weber County,29040.0,South Ogden,ACCOMMODATION (721),30000.0,True
2394,Weber County,29040.0,South Ogden,FOOD SERVICES & DRINKING PLACES (722),9660230.0,False
2395,Weber County,29040.0,South Ogden,OTHER SERVICES-EXCEPT PUBLIC ADMINISTRATION (81),1380064.0,False
2396,Weber County,29040.0,South Ogden,PRIVATE MOTOR VEHICLE SALES,1250000.0,True
2397,Weber County,29040.0,South Ogden,PRIOR-PERIOD PAYMENTS & REFUNDS,770844.0,False


In [7]:
# pivot the table to get sectors as columns with tax values by city
sectors_q1_2019 = pd.pivot_table(q1_2019,values = '2019Q1',index ='City', columns = 'Economic Sector (NAICS Code)', aggfunc='first')

# fill NAs with 0
sectors_q1_2019 = sectors_q1_2019.fillna(0)

# fix column names
#new_names = [replace(item) for item in list(sectors_q1_2019.columns)]
new_names = ['L_ACCOMMODATION_19', 'S_ADMIN_SUPPORT_19','A_AG_WILDLIFE_19','L_CULTURAL_REC_19','C_CONSTRUCTION_19',
'E_EDUCATION_19','S_FINANCIAL_19','L_RSTRNT_BAR_19','H_HEALTH_CARE_19','S_IT_DATA_19','S_CORPORATE_MGMT_19',
'M_MANUFACTURING_19','E_EXTRACTION_19','O_OTHER_19','X_ADJUSTMENTS_19','R_AUTO_PRIVATE_19','S_PROF_TECH_SRV_19',
'G_GOVERNMENT_19','S_RENTAL_LEASING_19','R_BUILDING_SUPPLY_19','R_CLOTHING_19','R_ELECTRONICS_19','R_GROCERY_BEV_19',
'R_FURNITURE_19','R_GAS_STATIONS_19','R_GENERAL_RETAIL_19','R_HEALTH_RETAIL_19','R_OTHER_RETAIL_19','R_AUTO_RETAIL_19',
'R_NONSTORE_RETAIL_19','R_SPORT_HOBBY_19','L_SPECIAL_EVENT_19','W_DISTRIBUTION_19','X_UNKNOWN_19','U_UTILITIES_19',
'W_WHLSALE_DURABLE_19','S_WHLSLE_ETRADE_19','W_WHLSLE_NDURABLE_19']
sectors_q1_2019.columns = new_names

# get the total sales tax revenue
sectors_q1_2019['TOTAL_19'] = sectors_q1_2019[new_names].sum(axis=1)

sectors_q1_2019.head()

,L_ACCOMMODATION_19,S_ADMIN_SUPPORT_19,A_AG_WILDLIFE_19,L_CULTURAL_REC_19,C_CONSTRUCTION_19,E_EDUCATION_19,S_FINANCIAL_19,L_RSTRNT_BAR_19,H_HEALTH_CARE_19,S_IT_DATA_19,S_CORPORATE_MGMT_19,M_MANUFACTURING_19,E_EXTRACTION_19,O_OTHER_19,X_ADJUSTMENTS_19,R_AUTO_PRIVATE_19,S_PROF_TECH_SRV_19,G_GOVERNMENT_19,S_RENTAL_LEASING_19,R_BUILDING_SUPPLY_19,R_CLOTHING_19,R_ELECTRONICS_19,R_GROCERY_BEV_19,R_FURNITURE_19,R_GAS_STATIONS_19,R_GENERAL_RETAIL_19,R_HEALTH_RETAIL_19,R_OTHER_RETAIL_19,R_AUTO_RETAIL_19,R_NONSTORE_RETAIL_19,R_SPORT_HOBBY_19,L_SPECIAL_EVENT_19,W_DISTRIBUTION_19,X_UNKNOWN_19,U_UTILITIES_19,W_WHLSALE_DURABLE_19,S_WHLSLE_ETRADE_19,W_WHLSLE_NDURABLE_19,TOTAL_19
City,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
American Fork,600000.0,619006.0,150000.0,1655844.0,6481514.0,89078.0,2616295.0,23486666.0,478232.0,9027053.0,15000.0,1629561.0,90000.0,10323341.0,2235225.0,3000000.0,3404294.0,15000.0,3494619.0,15701968.0,6160442.0,6990447.0,3872346.0,3623720.0,2349484.0,34160592.0,5603710.0,5780807.0,79458820.0,6551315.0,6840013.0,1000.0,450000.0,0.0,7750000.0,9692571.0,20000.0,5607870.0,270025833.0
Beaver City,838216.0,23482.0,15000.0,7000.0,80000.0,20000.0,198107.0,2071292.0,45000.0,697839.0,2000.0,683402.0,0.0,775040.0,242275.0,400000.0,65884.0,0.0,87252.0,400000.0,52603.0,21392.0,1500000.0,3000.0,2750000.0,885188.0,75129.0,248421.0,595716.0,492305.0,29920.0,0.0,2000.0,7000.0,900000.0,231521.0,20000.0,159634.0,14625618.0
Blanding,479889.0,33014.0,1000.0,25000.0,124403.0,35000.0,21600.0,400000.0,50000.0,835190.0,1000.0,128951.0,0.0,257825.0,162960.0,800000.0,81626.0,800000.0,102562.0,1264714.0,47176.0,10691.0,1750000.0,8000.0,1250000.0,1062138.0,56051.0,149761.0,269373.0,563908.0,28143.0,0.0,10000.0,0.0,150000.0,481479.0,1000.0,502661.0,11945115.0
Bountiful,200000.0,2472059.0,1000.0,1162941.0,1810519.0,115181.0,1163654.0,12483316.0,990965.0,5098392.0,30000.0,2888286.0,15000.0,6313370.0,2545449.0,4750000.0,2302467.0,15000.0,1979269.0,1210782.0,2492409.0,3692337.0,19843788.0,1608221.0,2250000.0,1612327.0,1774760.0,3032325.0,50087412.0,6609984.0,1277616.0,0.0,20000.0,0.0,11000000.0,4456579.0,9000.0,633219.0,157947627.0
Brigham,1250000.0,179287.0,1000.0,350000.0,2312924.0,52439.0,194102.0,6096734.0,348583.0,2900344.0,20000.0,4667069.0,1000.0,3207128.0,3453277.0,2000000.0,345233.0,1000.0,1121260.0,1767578.0,446126.0,1163721.0,9500000.0,493607.0,3000000.0,2175546.0,358659.0,1511549.0,12390972.0,2658050.0,287331.0,1000.0,100000.0,0.0,4250000.0,3142281.0,5000.0,322188.0,72074988.0


### Calculate the differences 
*2020 - 2019 = Diff*

In [8]:
# merge the 2019-2020 difference with the sdf
sectors_q1_diff = pd.DataFrame(sectors_q1_2020.values - sectors_q1_2019.values)

# add back the column names
new_names = ['L_ACCOMMODATION_D20', 'S_ADMIN_SUPPORT_D20','A_AG_WILDLIFE_D20','L_CULTURAL_REC_D20','C_CONSTRUCTION_D20',
'E_EDUCATION_D20','S_FINANCIAL_D20','L_RSTRNT_BAR_D20','H_HEALTH_CARE_D20','S_IT_DATA_D20','S_CORPORATE_MGMT_D20',
'M_MANUFACTURING_D20','E_EXTRACTION_D20','O_OTHER_D20','X_ADJUSTMENTS_D20','R_AUTO_PRIVATE_D20','S_PROF_TECH_SRV_D20',
'G_GOVERNMENT_D20','S_RENTAL_LEASING_D20','R_BUILDING_SUPPLY_D20','R_CLOTHING_D20','R_ELECTRONICS_D20','R_GROCERY_BEV_D20',
'R_FURNITURE_D20','R_GAS_STATIONS_D20','R_GENERAL_RETAIL_D20','R_HEALTH_RETAIL_D20','R_OTHER_RETAIL_D20','R_AUTO_RETAIL_D20',
'R_NONSTORE_RETAIL_D20','R_SPORT_HOBBY_D20','L_SPECIAL_EVENT_D20','W_DISTRIBUTION_D20','X_UNKNOWN_D20','U_UTILITIES_D20',
'W_WHLSALE_DURABLE_D20','S_WHLSLE_ETRADE_D20','W_WHLSLE_NDURABLE_D20', 'TOTAL_D20']

sectors_q1_diff.columns = new_names

# add back cities as index
sectors_q1_diff['City'] = sectors_q1_2020.index
sectors_q1_diff = sectors_q1_diff.set_index('City')

#check table
sectors_q1_diff.head(10)

,L_ACCOMMODATION_D20,S_ADMIN_SUPPORT_D20,A_AG_WILDLIFE_D20,L_CULTURAL_REC_D20,C_CONSTRUCTION_D20,E_EDUCATION_D20,S_FINANCIAL_D20,L_RSTRNT_BAR_D20,H_HEALTH_CARE_D20,S_IT_DATA_D20,S_CORPORATE_MGMT_D20,M_MANUFACTURING_D20,E_EXTRACTION_D20,O_OTHER_D20,X_ADJUSTMENTS_D20,R_AUTO_PRIVATE_D20,S_PROF_TECH_SRV_D20,G_GOVERNMENT_D20,S_RENTAL_LEASING_D20,R_BUILDING_SUPPLY_D20,R_CLOTHING_D20,R_ELECTRONICS_D20,R_GROCERY_BEV_D20,R_FURNITURE_D20,R_GAS_STATIONS_D20,R_GENERAL_RETAIL_D20,R_HEALTH_RETAIL_D20,R_OTHER_RETAIL_D20,R_AUTO_RETAIL_D20,R_NONSTORE_RETAIL_D20,R_SPORT_HOBBY_D20,L_SPECIAL_EVENT_D20,W_DISTRIBUTION_D20,X_UNKNOWN_D20,U_UTILITIES_D20,W_WHLSALE_DURABLE_D20,S_WHLSLE_ETRADE_D20,W_WHLSLE_NDURABLE_D20,TOTAL_D20
City,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
American Fork,-138654.0,-35478.0,50000.0,16648.0,2068027.0,-11549.0,576097.0,215062.0,-7296.0,-514647.0,-6000.0,2146185.0,-10000.0,-361365.0,-416087.0,-1000000.0,862124.0,0.0,-67237.0,1561091.0,-1153210.0,-508409.0,755565.0,-341707.0,150516.0,4184381.0,538092.0,800129.0,-3490141.0,7509073.0,-369956.0,-1000.0,109259.0,0.0,500000.0,4591672.0,285397.0,-1646996.0,16839586.0
Beaver City,25895.0,-1325.0,15000.0,1000.0,-60000.0,0.0,56710.0,-57544.0,15000.0,-262738.0,4000.0,474673.0,3000.0,13633.0,22633.0,-50000.0,50844.0,0.0,-6476.0,97496.0,-7520.0,7976.0,250000.0,1000.0,250000.0,-408770.0,16312.0,-26604.0,62975.0,783385.0,-9348.0,0.0,0.0,-7000.0,0.0,174065.0,-207.0,-31503.0,1396562.0
Blanding,-129546.0,1719.0,0.0,0.0,125597.0,115000.0,997.0,0.0,10000.0,-393960.0,0.0,111417.0,0.0,12096.0,-788810.0,-200000.0,21866.0,-100000.0,-42014.0,43263.0,4477.0,104425.0,250000.0,-5000.0,0.0,-416255.0,18500.0,-42438.0,-5004.0,647346.0,13678.0,0.0,20000.0,0.0,0.0,-178137.0,7000.0,-213945.0,-1007728.0
Bountiful,0.0,-374373.0,14000.0,-160211.0,54379.0,24741.0,27960.0,-191766.0,177429.0,1338191.0,0.0,-245827.0,0.0,141772.0,-895892.0,-1750000.0,961342.0,55000.0,-79088.0,-99108.0,-110784.0,-468323.0,2822493.0,-143718.0,0.0,194844.0,402022.0,-106336.0,-7175918.0,7598069.0,477812.0,0.0,30000.0,1000.0,250000.0,887150.0,91630.0,90214.0,3838704.0
Brigham,-391584.0,5169.0,2000.0,0.0,-1586913.0,-17.0,209991.0,-155792.0,-1993.0,-145235.0,-12000.0,617269.0,0.0,90854.0,-2867562.0,0.0,348908.0,0.0,440839.0,-245197.0,-77540.0,152522.0,1784680.0,38051.0,-117676.0,-1646498.0,12220.0,-385512.0,-90610.0,2423733.0,77702.0,0.0,-20000.0,0.0,0.0,5681302.0,121696.0,-39072.0,4223735.0
Cedar City,-202581.0,289464.0,1000.0,-26273.0,-558507.0,3728.0,-116105.0,149316.0,5729.0,785340.0,-10000.0,2976564.0,-25000.0,700949.0,234058.0,0.0,655653.0,-50000.0,531928.0,4813165.0,-387619.0,510010.0,2760361.0,166386.0,1158288.0,3995170.0,185841.0,1577690.0,-1389931.0,5925616.0,648076.0,18000.0,-1949726.0,0.0,1000000.0,550402.0,18785.0,696297.0,25642074.0
Centerville,-10000.0,43805.0,0.0,-9000.0,65253.0,3789.0,-347307.0,394203.0,-41064.0,-392471.0,-54000.0,1530905.0,-10000.0,503752.0,-606060.0,-350000.0,83382.0,0.0,191321.0,1156996.0,-158102.0,602623.0,1255704.0,1192043.0,250000.0,1646938.0,35385.0,-434514.0,-240029.0,2875806.0,-101838.0,0.0,-2000.0,0.0,0.0,-191012.0,-115631.0,105479.0,8874356.0
Clearfield,-23250.0,1737.0,0.0,1200000.0,450892.0,-2387.0,-25057.0,-123919.0,-611.0,377238.0,0.0,1888759.0,-3000.0,-200666.0,-8061986.0,0.0,1089088.0,-50000.0,404104.0,522466.0,46711.0,192215.0,-619848.0,-4921.0,891392.0,125923.0,29252.0,-329065.0,-436673.0,4498484.0,-116385.0,-6000.0,-20000.0,0.0,-1250000.0,917559.0,7307.0,193848.0,1563207.0
Cottonwood Heights,345039.0,464263.0,2000.0,-1100000.0,94155.0,269.0,-44971.0,-2858661.0,-50597.0,830941.0,0.0,-539569.0,-35000.0,97227.0,389057.0,-1500000.0,-422058.0,-50000.0,238005.0,-80954.0,-636508.0,72321.0,4270880.0,34718.0,0.0,-20838.0,665789.0,57809.0,-77165.0,6189222.0,-180962.0,-4000.0,3000.0,0.0,750000.0,334464.0,52582.0,213742.0,7504200.0


In [9]:
# Check join output shape
print(sectors_q1_2019.shape)
print(sectors_q1_2020.shape)
print(sectors_q1_diff.shape)

(66, 39)
(66, 39)
(66, 39)


### Load municipalities and townships shapefile and join to formatted sales tax data

In [10]:
# load cities shapefile into pandas spatial dataframe
cities_shp = '.\\Inputs\\Cities.shp'
cities_sdf = pd.DataFrame.spatial.from_featureclass(cities_shp)

# Rename columns
cities_sdf.columns = ['FID', 'NAME', 'SOURCE', 'SALESTAXID', 'POPLASTCEN', 'POPLASTEST',
       'AREA_SQMI', 'SHAPE']

In [11]:
# merge the 2020 sdf with the pivoted data
cities_sdf2 = cities_sdf.merge(sectors_q1_2019, left_on= "NAME", right_on="City", how="inner")
cities_sdf2 = cities_sdf2.merge(sectors_q1_2020, left_on= "NAME", right_on="City", how="inner")
cities_sdf2 = cities_sdf2.merge(sectors_q1_diff, left_on= "NAME", right_on="City", how="inner")
cities_sdf2.tail(10)


,FID,NAME,SOURCE,SALESTAXID,POPLASTCEN,POPLASTEST,AREA_SQMI,SHAPE,L_ACCOMMODATION_19,S_ADMIN_SUPPORT_19,A_AG_WILDLIFE_19,L_CULTURAL_REC_19,C_CONSTRUCTION_19,E_EDUCATION_19,S_FINANCIAL_19,L_RSTRNT_BAR_19,H_HEALTH_CARE_19,S_IT_DATA_19,S_CORPORATE_MGMT_19,M_MANUFACTURING_19,E_EXTRACTION_19,O_OTHER_19,X_ADJUSTMENTS_19,R_AUTO_PRIVATE_19,S_PROF_TECH_SRV_19,G_GOVERNMENT_19,S_RENTAL_LEASING_19,R_BUILDING_SUPPLY_19,R_CLOTHING_19,R_ELECTRONICS_19,R_GROCERY_BEV_19,R_FURNITURE_19,R_GAS_STATIONS_19,R_GENERAL_RETAIL_19,R_HEALTH_RETAIL_19,R_OTHER_RETAIL_19,R_AUTO_RETAIL_19,R_NONSTORE_RETAIL_19,R_SPORT_HOBBY_19,L_SPECIAL_EVENT_19,W_DISTRIBUTION_19,X_UNKNOWN_19,U_UTILITIES_19,W_WHLSALE_DURABLE_19,S_WHLSLE_ETRADE_19,W_WHLSLE_NDURABLE_19,TOTAL_19,L_ACCOMMODATION_20,S_ADMIN_SUPPORT_20,A_AG_WILDLIFE_20,L_CULTURAL_REC_20,C_CONSTRUCTION_20,E_EDUCATION_20,S_FINANCIAL_20,L_RSTRNT_BAR_20,H_HEALTH_CARE_20,S_IT_DATA_20,S_CORPORATE_MGMT_20,M_MANUFACTURING_20,E_EXTRACTION_20,O_OTHER_20,X_ADJUSTMENTS_20,R_AUTO_PRIVATE_20,S_PROF_TECH_SRV_20,G_GOVERNMENT_20,S_RENTAL_LEASING_20,R_BUILDING_SUPPLY_20,R_CLOTHING_20,R_ELECTRONICS_20,R_GROCERY_BEV_20,R_FURNITURE_20,R_GAS_STATIONS_20,R_GENERAL_RETAIL_20,R_HEALTH_RETAIL_20,R_OTHER_RETAIL_20,R_AUTO_RETAIL_20,R_NONSTORE_RETAIL_20,R_SPORT_HOBBY_20,L_SPECIAL_EVENT_20,W_DISTRIBUTION_20,X_UNKNOWN_20,U_UTILITIES_20,W_WHLSALE_DURABLE_20,S_WHLSLE_ETRADE_20,W_WHLSLE_NDURABLE_20,TOTAL_20,L_ACCOMMODATION_D20,S_ADMIN_SUPPORT_D20,A_AG_WILDLIFE_D20,L_CULTURAL_REC_D20,C_CONSTRUCTION_D20,E_EDUCATION_D20,S_FINANCIAL_D20,L_RSTRNT_BAR_D20,H_HEALTH_CARE_D20,S_IT_DATA_D20,S_CORPORATE_MGMT_D20,M_MANUFACTURING_D20,E_EXTRACTION_D20,O_OTHER_D20,X_ADJUSTMENTS_D20,R_AUTO_PRIVATE_D20,S_PROF_TECH_SRV_D20,G_GOVERNMENT_D20,S_RENTAL_LEASING_D20,R_BUILDING_SUPPLY_D20,R_CLOTHING_D20,R_ELECTRONICS_D20,R_GROCERY_BEV_D20,R_FURNITURE_D20,R_GAS_STATIONS_D20,R_GENERAL_RETAIL_D20,R_HEALTH_RETAIL_D20,R_OTHER_RETAIL_D20,R_AUTO_RETAIL_D20,R_NONSTORE_RETAIL_D20,R_SPORT_HOBBY_D20,L_SPECIAL_EVENT_D20,W_DISTRIBUTION_D20,X_UNKNOWN_D20,U_UTILITIES_D20,W_WHLSALE_DURABLE_D20,S_WHLSLE_ETRADE_D20,W_WHLSLE_NDURABLE_D20,TOTAL_D20
55,222,Syracuse,Cities,049,25113,30400,10.193900,"{'rings': [[[411406.60675760085, 4549994.30436...",30000.0,69731.0,1000.0,800000.0,170692.0,27490.0,374231.0,3978564.0,25000.0,2779301.0,7000.0,446316.0,20000.0,1214619.0,524698.0,3250000.0,272981.0,4000.0,871403.0,376848.0,310827.0,782287.0,7250000.0,9950400.0,400000.0,15526207.0,369963.0,268781.0,366701.0,4018725.0,91521.0,0.0,4000.0,50000.0,5250000.0,1017122.0,9000.0,206649.0,61116057.0,40000.0,63742.0,3000.0,900000.0,321930.0,26492.0,426206.0,4237504.0,40000.0,2751481.0,9000.0,393712.0,15000.0,1413903.0,394802.0,2500000.0,860434.0,15000.0,966102.0,510692.0,436000.0,657970.0,10750000.0,9529395.0,450000.0,18441622.0,445481.0,505804.0,648784.0,8973807.0,192182.0,0.0,20000.0,0.0,5000000.0,1359640.0,37511.0,221366.0,73558562.0,10000.0,-5989.0,2000.0,100000.0,151238.0,-998.0,51975.0,258940.0,15000.0,-27820.0,2000.0,-52604.0,-5000.0,199284.0,-129896.0,-750000.0,587453.0,11000.0,94699.0,133844.0,125173.0,-124317.0,3500000.0,-421005.0,50000.0,2915415.0,75518.0,237023.0,282083.0,4955082.0,100661.0,0.0,16000.0,-50000.0,-250000.0,342518.0,28511.0,14717.0,12442505.0
56,224,Taylorsville,Cities,142,59989,60192,10.839700,"{'rings': [[[422191.826826254, 4504303.3642982...",300000.0,493259.0,8000.0,1440305.0,879035.0,79501.0,1971955.0,25781931.0,158751.0,9067000.0,25000.0,2033771.0,25000.0,2076728.0,2913091.0,4000000.0,1452652.0,0.0,1689952.0,378895.0,5060457.0,1884803.0,13965824.0,441957.0,3250000.0,24804388.0,1039078.0,4465042.0,6659493.0,5760021.0,6442738.0,8000.0,45000.0,0.0,11500000.0,4439978.0,20000.0,453177.0,145014782.0,400000.0,503239.0,10000.0,1249110.0,1306103.0,121591.0,2283612.0,25553638.0,137265.0,8254351.0,15000.0,2111268.0,20000.0,1873257.0,1277841.0,3000000.0,2671435.0,2000.0,2201892.0,292650.0,4531724.0,4690724.0,16507857.0,384151.0,3500000.0,25332716.0,1298342.0,4612787.0,6676386.0,12777982.0,

In [12]:
# Confirm join count, Salt Lake County (Unincorporated) should be the only one not joined
l1 = list(q1_2020['City'].value_counts().index)
l2 = list(cities_sdf['NAME'].value_counts().index)
list(set(l1) - set(l2))

['Salt Lake County (Unincorporated)']

In [13]:
# Create file gdb and export sdf to feature class
outputs = '.\\Outputs'

gdb = os.path.join(outputs, "taxable_sales.gdb")
if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs, "taxable_sales.gdb")

cities_sdf2.spatial.to_featureclass(location=os.path.join(outputs, "taxable_sales.gdb","taxable_sales_utah_q1"))

'E:\\Projects\\UT_Sales_Tax_Analysis\\Outputs\\taxable_sales.gdb\\taxable_sales_utah_q1'

### Analysis

In [14]:
# Get year specific column names
standard_cols = ['fid','name','source','salestaxid','poplastcen','poplastcen','area_sqmi', 'SHAPE']
names_19 = standard_cols + [col for col in cities_sdf2.columns if '_19' in col]
names_20 = standard_cols + [col for col in cities_sdf2.columns if '_20' in col]
names_D20 = standard_cols + [col for col in cities_sdf2.columns if '_d20' in col]

# subset columns by time
cities_q1_19 = cities_sdf2[names_19]
cities_q1_20 = cities_sdf2[names_20]
cities_q1_d20 = cities_sdf2[names_D20]


In [15]:
# 10 cities with smallest difference of revenue
cities_q1_d20[['name', 'poplastcen', 'area_sqmi', 'total_d20']].sort_values('total_d20', ascending=False).head(10)

,name,poplastcen,poplastcen,area_sqmi,total_d20
45,Salt Lake City,190749,190749,110.734001,95830211.0
54,St George,76548,76548,77.209900,61967355.0
50,South Salt Lake,24282,24282,6.936550,51705330.0
63,West Valley City,133796,133796,35.799702,40029181.0
21,Lehi,49315,49315,28.898199,36237576.0
48,South Jordan,51057,51057,22.263201,35667478.0
62,West Jordan,105891,105891,32.306999,33691702.0
51,Spanish Fork,36232,36232,16.249701,30567441.0
18,Kaysville,28050,28050,10.645300,27736795.0
26,Millcreek,62139,62139,12.916500,26248619.0


In [16]:
# 10 cities with largest difference of revenue
cities_q1_d20[['name', 'poplastcen', 'area_sqmi', 'total_d20']].sort_values('total_d20', ascending=True).head(10)

,name,poplastcen,poplastcen,area_sqmi,total_d20
35,Park City,7709,7709,20.34170,-35833502.0
59,Vernal,9460,9460,4.63271,-9388974.0
30,Murray,47311,47311,12.29370,-9197282.0
27,Moab,5124,5124,4.80246,-6035165.0
43,Roosevelt,6260,6260,6.75707,-3787091.0
52,Springdale,569,569,4.59488,-2544861.0
12,Farmington,18752,18752,10.03510,-2069172.0
46,Sandy,88693,88693,24.13690,-1503651.0
2,Blanding,3514,3514,13.20550,-1007728.0
32,North Salt Lake,16930,16930,8.51215,-649369.0


#### Some industries of interest:
- RETAIL-GASOLINE STATIONS (447)
- RETAIL-FOOD & BEVERAGE STORES (445)
- REAL ESTATE, RENTAL & LEASING (53)
- ARTS, ENTERTAINMENT AND RECREATION (71)
- ACCOMMODATION (721)